# Produce tables of stage 2 MWAS results

In [1]:
library(data.table)

In [2]:
getwd()

[1] "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/scripts"

In [3]:
df <- fread("12-OUT_matched_SNP_meth_cov_outputs.csv")

In [ ]:
import os
import pandas as pd
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.methods import RS4

# Enable the automatic conversion of pandas data frames to R data frames and vice versa
pandas2ri.activate()

# Import R packages
base = importr('base')
data_table = importr('data.table')
readRDS = base.readRDS

# Read the CSV file
df = data_table.fread("12-OUT_matched_SNP_meth_cov_outputs.csv").to_pandas()

traits = ["bp", "mdd", "scz"]

for trait in traits:
    print(trait)
    combined_data = []

    df['stage2_path'] = df['path'].apply(lambda x: x.replace('.rds', f'_gwas_stat_{trait}_results.rds'))

    for i in range(len(df)):
        print(i)
        stage2_in = readRDS(df.loc[i, 'stage2_path'])
        stage1_in = readRDS(df.loc[i, 'path'])

        if len(stage1_in.rx2('models')) != len(stage2_in.rx2('MWASmodels')):
            raise ValueError("Files don't match")

        for j in range(len(stage1_in.rx2('models'))):
            model1 = stage1_in.rx2('models')[j]
            model2 = stage2_in.rx2('MWASmodels')[j]

            this_z = model2.rx2('z')[0]
            this_p = model2.rx2('p')[0]
            this_n = model2.rx2('n')[0]
            this_pos = model1.rx2('methylationPosition')[0]
            this_stats = stage2_in.rx2('summary_stats_path')[0]
            this_scaff = stage1_in.rx2('scaffoldIdentifier')[0]

            combined_data.append({
                'z': this_z,
                'p': this_p,
                'n': this_n,
                'pos': this_pos,
                'stats': this_stats,
                'scaff': this_scaff
            })

    combined_df = pd.DataFrame(combined_data)
    combined_df.to_csv(f"16a3-OUT_stage2_MWAS_{trait}.csv", index=False)


In [ ]:
this_p

In [ ]:
this_z

In [ ]:
this_ 